## Advanced Lane Finding + Vehicle Detection Project

This project is a combination of Advanced Lane Finding and Vehicle Detection.



* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

In [ ]:
import importlib
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from roadlanes import draw_lane
import roadlanes
import adv_lane_lines
import searchlines
import perspectivetransform
import correctcamera
import object_detection_findmatches
from glob import glob
import object_detection_utils
import object_detection_colorspaces
import object_detection_fecvector
import object_detection_car_notcar

In [ ]:
importlib.reload(adv_lane_lines)
importlib.reload(roadlanes)
importlib.reload(searchlines)
importlib.reload(perspectivetransform)
importlib.reload(correctcamera)

# Object Finding - using various methods

## This method uses historgrams of different colors to identify a pattern characteristic for an object

In [ ]:
importlib.reload(object_detection_utils)
object_detection_utils.main_histogram()

## Using color spaces and clustering to discover objects

In [ ]:
import object_detection_colorspaces
importlib.reload(object_detection_colorspaces)
object_detection_colorspaces.main_color_spaces()

## Object detection using template matching

In [ ]:
import object_detection_findmatches
importlib.reload(object_detection_findmatches)
object_detection_findmatches.main_drawboxes()

In [ ]:
## using feature vector approach for pictures of lower resolution

In [ ]:
import object_detection_fecvector
importlib.reload(object_detection_fecvector)
object_detection_fecvector.main_fecvector()

In [ ]:
## Image dataset analysis: cars vs. non-cars

In [ ]:
import object_detection_car_notcar
importlib.reload(object_detection_car_notcar)
object_detection_car_notcar.main_look_for_car()

In [ ]:
import object_detection_hog_features

# Image data Exploration

# Let's see what kind of images we have for vehicles and non-vehicle objects

In [ ]:
car_images = glob('util_images/vehicles/**/*.png')
noncar_images = glob('util_images/non-vehicles/**/*.png')

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(16, 16))
axs = axs.ravel()

for i in np.arange(8):
    img = cv2.imread(noncar_images[np.random.randint(0,len(noncar_images))])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    axs[i].axis('off')
    axs[i].set_title('a non-vehicle object', fontsize=10)
    axs[i].imshow(img)

for i in np.arange(8,16):
    img = cv2.imread(car_images[np.random.randint(0,len(car_images))])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    axs[i].axis('off')
    axs[i].set_title('a vehicle', fontsize=10)
    axs[i].imshow(img)
    


In [ ]:
print("Number of vehicle images: {}".format(len(car_images)))
print("Number of non-vehicle images: {}".format(len(noncar_images)))

# Exploring various models and choose the best one

## Calculation of models...

In [ ]:
import object_detection_hog_features
importlib.reload(object_detection_hog_features)
experiments_dict = object_detection_hog_features.main_hog()

## Displaying results of calculations for various models...

In [ ]:
importlib.reload(object_detection_hog_features)
object_detection_hog_features.print_results(experiments_dict)

## Getting results better than specified accuracy

In [ ]:
# getting models with accuacy 0.99 or higher
results_best_models = object_detection_hog_features.get_models(experiments_dict, accuracy=0.991)
results_best_model = object_detection_hog_features.get_best_model(experiments_dict)
object_detection_hog_features.print_results({"best_model": results_best_model})

# Extracing HOG (Histogram of Oriented Gradients) features for 2 examples

In [ ]:
car_img = mpimg.imread(car_images[5])
_, car_dst = object_detection_hog_features.get_hog_features(car_img[:,:,2], 9, 8, 8, vis=True, feature_vec=True)
noncar_img = mpimg.imread(noncar_images[5])
_, noncar_dst = object_detection_hog_features.get_hog_features(noncar_img[:,:,2], 9, 8, 8, vis=True, feature_vec=True)

# Visualize 
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(7,7))
f.subplots_adjust(hspace = .4, wspace=.2)
ax1.imshow(car_img)
ax1.set_title('Car Image', fontsize=16)
ax2.imshow(car_dst, cmap='gray')
ax2.set_title('Car HOG', fontsize=16)
ax3.imshow(noncar_img)
ax3.set_title('Non-Car Image', fontsize=16)
ax4.imshow(noncar_dst, cmap='gray')
ax4.set_title('Non-Car HOG', fontsize=16)

In [ ]:
import object_detection_hog_subsumpling
importlib.reload(object_detection_hog_features)
importlib.reload(object_detection_hog_subsumpling)

In [ ]:
test_img = mpimg.imread('./test_images/test1.jpg')

ystart = 400
ystop = 656
scale = 1.5

colorspace = results_best_model['color_item']
orient = results_best_model['orient']
pix_per_cell = results_best_model['pix_per_cell']
cell_per_block = results_best_model['cell_per_block']
hog_channel = results_best_model['hog_channel']
svc = results_best_model['linearSVC']
x_scaler = results_best_model["X_scaler"]
spatial_size = results_best_model["spatial_size"]
hist_bins = results_best_model["hist_bins"]

rectangles = object_detection_hog_subsumpling.find_cars(test_img, ystart, ystop, scale, colorspace, hog_channel, svc, x_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)

print(len(rectangles), 'rectangles found in image')

In [ ]:
import object_detection_sliding_window
importlib.reload(object_detection_utils)
importlib.reload(object_detection_sliding_window)
object_detection_sliding_window.sliding_window_main()